In [22]:
import ee
import folium

# opens browser to request authenticatication token (OAUTH2) and store it.
#ee.Authenticate()

# uses your stored auth token to login to ee.
ee.Initialize()



Successfully saved authorization token.


In [23]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. Here
    self is a folium.Map class instance, and we are adding 
    additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self

In [4]:
# create an ee vector object
region = ee.Feature(
    ee.Geometry.Polygon([
        [-100, 40],
        [-90, 40],
        [-100, 50],
    ])
)

# create a map
emap = EngineMap()

# add ee vector feature
emap.add_ee_vector(region)

# focus location/bounds of the map on its child features
emap.fit_bounds(emap.get_bounds(), max_zoom=5)
emap

In [60]:
# load Hansen dataset
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# create a map
emap = EngineMap()

# add raster to map
emap.add_ee_raster(gfc2014, {})
emap.add_ee_raster(gfc2014, {'bands': ['treecover2000']})
emap.add_ee_raster(gfc2014, {'bands' : ['loss', 'treecover2000', 'gain'],
                            'max' : [1, 255, 1]})
emap.add_ee_raster(gfc2014.updateMask(gfc2014), {'bands' : ['last_b50', 'last_b40', 'last_b30']})
        
emap

In [32]:
emap.add_ee_raster(gfc2014, {'bands' : ['treecover2000'],
                            'palette' : ['000000', '00FF00'],
                            'max' : 100
                            })
                            
emap

In [53]:
gfc2014M = gfc2014.updateMask(gfc2014)

emap.add_ee_raster(gfc2014M, {'bands' : ['treecover2000'],
                            'palette' : ['000000', '00FF00'],
                            'max' : 100
                            })
emap

In [71]:
treeCover = gfc2014.select(['treecover2000'])
lossImage = gfc2014.select(['loss'])
gainImage = gfc2014.select(['gain'])

gainAndLoss = gainImage.And(lossImage)

fmap=EngineMap(location=[38, -76], zoom_start=8)

fmap.add_ee_raster(
    treeCover.updateMask(treeCover),
    {'palette' : ['000000','00FF00']}
    ).add_ee_raster(
    lossImage.updateMask(lossImage),
    {'palette' : ['FF0000']}    
    ).add_ee_raster(
    gainImage.updateMask(gainImage),
    {'palette' : ['0000FF']}
    ).add_ee_raster(
    gainAndLoss.updateMask(gainAndLoss),
    {'palette': 'FF00FF'},
    attr = "Alice"
    )
